In [45]:
%%time
import requests, psycopg2, random, os, hashlib
from dotenv import load_dotenv
from datetime import date

DB_ENV_PROD=0
load_dotenv()
api_key=os.getenv('API_KEY')

if DB_ENV_PROD==0:
    database=os.getenv('LOCAL_DB_NAME')
    user=os.getenv('LOCAL_DB_USER')
    password=os.getenv('LOCAL_DB_PASSWORD')
    port=int(os.getenv('DB_PORT'))
    host=os.getenv('DB_HOST')

    def database_connection(host=host,database=database,user=user,password=password,port=port):
        conn = psycopg2.connect(host=host, database=database, user=user, password=password, port=port)
        return conn

    conn = database_connection()
    cur = conn.cursor()

elif DB_ENV_PROD==1:    
    REMOTE_HOST = os.getenv('REMOTE_HOST')
    REMOTE_USERNAME = os.getenv('REMOTE_USERNAME')
    PKEY_PATH= './humble.pem'

    conn_params = {
        'database': os.getenv('CLOUD_DB_NAME'), 
        'user': os.getenv('CLOUD_DB_USER'), 
        'password': os.getenv('CLOUD_DB_PASSWORD'), 
        'host': os.getenv('CLOUD_DB_HOST'), 
        'port': int(os.getenv('CLOUD_DB_PORT'))
    }
    
    def database_connection():
        conn = psycopg2.connect(**conn_params)
        return conn
    
    conn = database_connection()
    cur = conn.cursor()


endpoints = ['actives', 'losers', 'gainers']

for endpoint in endpoints:
    url = f'https://fmpcloud.io/api/v3/{endpoint}?apikey={api_key}'
    res = requests.get(url).json()
    
    # TABLE CREATION
    command=f'''
        CREATE TABLE IF NOT EXISTS stock_market_{endpoint}_prices 
        (hashed_key text NOT NULL PRIMARY KEY, ticker text, changes float(24), price float(24), changesPercentage text, companyName text);
    '''
    try:
        cur.execute(command)
        conn.commit()
    except:
        conn.close()
        conn = database_connection()
        cur = conn.cursor()
        cur.execute(command)
        conn.commit()
    
    for record in res:
        data = list(record.values())
        data.insert(0, hashlib.sha256(data[0].encode('utf-8')).hexdigest())
        
        insert_query = f'''
            INSERT INTO stock_market_{endpoint}_prices (hashed_key, ticker, changes, price, changesPercentage, companyName)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (hashed_key) DO UPDATE SET
            (ticker, changes, price, changesPercentage, companyName) = (EXCLUDED.ticker, EXCLUDED.changes, EXCLUDED.price, EXCLUDED.changesPercentage, EXCLUDED.companyName);
        '''

        try:
            cur.execute(insert_query, data)
            conn.commit()
        except:
            conn.close()
            conn=database_connection()
            cur = conn.cursor()
            cur.execute(insert_query, data)
            conn.commit()
        
    print(f'{endpoint} <-- ADDED')
    

####################################################################################################
    
sectors_performance_url = f'https://fmpcloud.io/api/v3/sectors-performance?apikey={api_key}'
response = requests.get(sectors_performance_url).json()

# TABLE CREATION
command=f'''
    CREATE TABLE IF NOT EXISTS sectors_performance 
    (hashed_key text NOT NULL PRIMARY KEY, sector text, changesPercentage text);
'''
try:
    cur.execute(command)
    conn.commit()
except:
    conn.close()
    conn = database_connection()
    cur = conn.cursor()
    cur.execute(command)
    conn.commit()
    
for record in response:
    data = list(record.values())
    data.insert(0, hashlib.sha256(data[0].encode('utf-8')).hexdigest())

    insert_query = f'''
        INSERT INTO sectors_performance (hashed_key, sector, changesPercentage)
        VALUES (%s, %s, %s)
        ON CONFLICT (hashed_key) DO UPDATE SET
        (sector, changesPercentage) = (EXCLUDED.sector, EXCLUDED.changesPercentage);
    '''

    try:
        cur.execute(insert_query, data)
        conn.commit()
    except:
        conn.close()
        conn=database_connection()
        cur = conn.cursor()
        cur.execute(insert_query, data)
        conn.commit()
print('sectors_performance table updated')


####################################################################################################

stock_news_url = f'https://fmpcloud.io/api/v3/stock_news?limit=50&apikey={api_key}'
response = requests.get(stock_news_url).json()

# TABLE CREATION
command=f'''
    CREATE TABLE IF NOT EXISTS stock_news 
    (hashed_key text NOT NULL PRIMARY KEY, symbol text, published_date TIMESTAMP, title text, image text, size text, text_desc text, url text);
'''
try:
    cur.execute(command)
    conn.commit()
except:
    conn.close()
    conn = database_connection()
    cur = conn.cursor()
    cur.execute(command)
    conn.commit()
    
for record in response:
    data = list(record.values())
    data.insert(0, hashlib.sha256(data[0].encode('utf-8')).hexdigest())

    insert_query = f'''
        INSERT INTO stock_news (hashed_key, symbol, published_date, title, image, size, text_desc, url)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (hashed_key) DO UPDATE SET
        (symbol, published_date, title, image, size, text_desc, url) = (EXCLUDED.symbol, EXCLUDED.published_date, EXCLUDED.title, EXCLUDED.image, EXCLUDED.size, EXCLUDED.text_desc, EXCLUDED.url);
    '''

    try:
        cur.execute(insert_query, data)
        conn.commit()
    except:
        conn.close()
        conn=database_connection()
        cur = conn.cursor()
        cur.execute(insert_query, data)
        conn.commit()
        
print('stock_news table updated')

actives <-- ADDED
losers <-- ADDED
gainers <-- ADDED
sectors_performance table updated
stock_news table updated
CPU times: total: 31.2 ms
Wall time: 1min 24s
